In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes

AutoModelForCausalLM --> to get the models; AutoTokens --> word to tokens(numbers) TrainingArguments --> what are arguments used to train Trainer --> auto trainer to train the model get_peft_model --> to get the model from peft LoraConfig --> setting the configuration of peft model TaskType --> what task we are going to do

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset

In [ ]:
data = {
    "text": [
        "###Question : what is planet?\n### Answer : planets are in solar system and each planet has its own characteristics like earth is a planet where humans live",

        "###Question : what is Galaxy?\n### Answer : a galaxy is a huge system of stars, dust, gas, and dark matter bound together by gravity; our solar system is part of the Milky Way galaxy",

        "###Question : what is the sun?\n### Answer : the sun is a massive star at the center of the solar system that provides light and heat to all the planets orbiting around it",

        "###Question : what is the moon?\n### Answer : the moon is earth’s only natural satellite and it reflects sunlight; it affects tides and has different phases like full moon and new moon",

        "###Question : what is a black hole?\n### Answer : a black hole is a region in space where gravity is so strong that nothing, not even light, can escape from it"
    ]
}


dataset = Dataset.from_dict(data)

In [ ]:
model_name = "tiiuae/falcon-rw-1b"
tokenizer=AutoTokenizer.from_pretrained(model_name) # setting tokenizer in the model, only then the input are automatically get tokenized and stored.
tokenizer.pad_token=tokenizer.eos_token #padding


Trainer always ask for labels, here we have only text and the label here is the previuos words in the sentence.

input_ids is the order of the word in the output to remeber and retain the sequence

In [ ]:
# def tokenize(batch):
#   tokens = tokenizer(
#       batch["text"], # input
#       padding = "max_length", # if a model size is 30 but input is 20 the padding will be fill remaining 10 as 0
#       truncation = True, # if the max length is reached it will truncate
#       max_length = 128,
#   )
#   tokens["labels"] = tokens["input_ids"].copy() # labels will have what sequence is the sentence and labels are the one from prevoius words
#   return tokens

# # tokenized_dataset = dataset.map(tokenize, btached = True) # entire dataset will fitted adn label will be assigned



# exact tokenizing if the input
def tokenize(batch):
  tokens=tokenizer(
      batch['text'],
      padding="max_length",
      truncation=True,
      max_length=128
  )
  tokens["labels"]=tokens["input_ids"].copy()
  return tokens



In [ ]:
tokenized_dataset =dataset.map(tokenize, batched=True) # entire dataset will fitted adn label will be assigned

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
# to fit the all above settings to the model

# setting model

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit = True,
    device_map = "auto" # automatically shifting between cpu and gpu based on requirements
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
lora_config = LoraConfig(
    r = 8, # adapters for finetuning the in diffrent places
    lora_alpha = 16, # scaling
    target_modules = ["query_key_value"],# attention mechanism only take impt nodes i.e is 1(one) and here is where the lora is working
    lora_dropout = 0.1, # while triainig to reduce the overfitting dropoutting the nodes
    bias = "none", # to calculate error
    task_type = TaskType.CAUSAL_LM # casual language modeling
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [ ]:
training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 3, # epochs
    per_device_train_batch_size = 1,
    learning_rate = 2e-4, # usually if the the accuracy is low the learning rate should be modifiead because we should know ifnthe model is learning correctly or not
    logging_steps = 1,# doing in a step by step by manner
    save_strategy = "no",
    report_to = "none"
)


In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    tokenizer = tokenizer
)

/tmp/ipython-input-37-1862931468.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss
1,5.304500
2,4.793400
3,3.821500
4,4.166500
5,3.241300
6,2.048800
7,2.119000
8,1.522600
9,1.376100
10,1.379700


TrainOutput(global_step=15, training_loss=2.3296792268753053, metrics={'train_runtime': 6.731, 'train_samples_per_second': 2.228, 'train_steps_per_second': 2.228, 'total_flos': 13941221621760.0, 'train_loss': 2.3296792268753053, 'epoch': 3.0})

In [ ]:
model.save_pretrained("lora-finetuned-model")
tokenizer.save_pretrained("lora-finetuned-tokenized") # tokenizing and saving

('lora-finetuned-tokenized/tokenizer_config.json',
 'lora-finetuned-tokenized/special_tokens_map.json',
 'lora-finetuned-tokenized/vocab.json',
 'lora-finetuned-tokenized/merges.txt',
 'lora-finetuned-tokenized/added_tokens.json',
 'lora-finetuned-tokenized/tokenizer.json')

it can be used along with the langchain